In [1]:
import numpy as np
import pandas as pd
import easydatascience as eds

import os
os.chdir('C:\\Users\\vucin\\Desktop\\The problem with Serbian economy\\1.Data Cleaning')

#For easier export
datasets = []

## 1. Household Income and Spending

In [6]:
income_hh = pd.read_csv('01monthly_average_household_incomes.csv', sep=';')
spending_hh = pd.read_csv('01monthly_average_household_spendings.csv', sep=';')

#Renaming and dropping columns
rename = {'nTer':'Region', 'nTipNaselja':'SettlementType', 'god':'Year', 'nVrPod':'SortMeOut', 
          'vrednost':'Amount(RSD)'}
drop = ['idindikator', 'IDTer', 'IDTipNaselja', 'mes', 'IDVrPod']

income_hh = income_hh.rename(columns=rename).rename(columns={'nRaspolozivaSredstva':'TypeOfIncome'})
income_hh = income_hh.drop(drop+['IDRaspolozivaSredstva'], axis=1)
spending_hh = spending_hh.rename(columns=rename).rename(columns={'nCOICOP':'TypeOfSpending'})
spending_hh = spending_hh.drop(drop+['IDCOICOP'], axis=1)

#Moving structural percent that was a row and should have been a column
struct_pct = income_hh.iloc[1::2]['Amount(RSD)'].reset_index(drop=True)
income_hh = income_hh.iloc[::2].reset_index(drop=True)
income_hh['StructuralPercentage'] = struct_pct
income_hh = income_hh.drop('SortMeOut', axis=1)

struct_pct = spending_hh.iloc[1::2]['Amount(RSD)'].reset_index(drop=True)
spending_hh = spending_hh.iloc[::2].reset_index(drop=True)
spending_hh['StructuralPercentage'] = struct_pct
spending_hh = spending_hh.drop('SortMeOut', axis=1)

#Filling zeroes
income_hh = income_hh.fillna(0)

#Translating
spendings = [['Укупно', 'Total'], ['Храна и безалкохолна пића', 'Food and Non-Alcoholic Beverages'],
             ['Алкохолна пића, дуван и наркотици', 'Alcohol, Tobacco and Narcotics'], ['Одећа и обућа', 'Clothing and Footwear'],
             ['Становање, вода, ел. енергија, гас и остала горива', 'Housing, Water, Energy, Gas, etc.'],
             ['Опрема за стан и текуће одржавање', 'Home Equipment and Maintenance'], ['Здравље', 'Health'], ['Транспорт', 'Transport'],
             ['Комуникације', 'Communication'], ['Рекреација и култура', 'Culture and Recreation'], ['Образовање', 'Education'],
             ['Ресторани и хотели', 'Hotels and Restaurants'], ['Остали лични предмети и остале услуге', 'Other Items and Services']]
for i in spendings:
   spending_hh.loc[spending_hh['TypeOfSpending']==i[0], 'TypeOfSpending'] = i[1]
spending_hh = spending_hh[spending_hh.TypeOfSpending != 'Drop']
    
incomes = [['Приходи домаћинстава - укупно', 'Total Earnings'], 
           ['Приходи домаћинстава у новцу', 'Earnings in Currency'],
           ['Приходи из редовног радног односа', 'Earnings from Orthodox Work Relations a.k.a. Job'],
           ['Приходи ван редовног радног односа', 'Earnings from Unorthodox Work Relations'],
           ['Пензије (старосне, породичне, инвалидске и остале)', 'Pensions'],
           ['Остала примања од социјалног осигурања', 'Social Sequrity Earnings'],
           ['Приходи од пољопривреде, лова и риболова', 'Earnings from Agricultural, Hunting and Fishing Actions'],
           ['Приходи из иностранства', 'Foreign Earnings'], ['Приходи од имовине', 'Real Estate Earnings'],
           ['Поклони и добици', 'Presents'], ['Потрошачки и инвестициони кредити', 'Investment Interest'],
           ['Остала примања', 'Rest'], ['Приходи домаћинстава у натури', 'Earnings in Labor'],
           ['Приходи у натури на име зараде', 'Other Earnings in Labor'], ['Натурална потрошња', 'Drop']]
for i in incomes:
    income_hh.loc[income_hh['TypeOfIncome']==i[0], 'TypeOfIncome'] = i[1]
income_hh = income_hh[income_hh.TypeOfIncome != 'Drop']

settlements = [['Укупно', 'Total'], ['Градска насеља', 'City'], ['Остало', 'Rest']]
for i in settlements:
    income_hh.loc[income_hh['SettlementType']==i[0], 'SettlementType'] = i[1]
    spending_hh.loc[spending_hh['SettlementType']==i[0], 'SettlementType'] = i[1]
    
regions = [['РЕПУБЛИКА СРБИЈА', 'State'], ['Београдски регион', 'Belgrade'], 
           ['Регион Војводине', 'Voyvodina/North'], 
           ['Регион Шумадије и Западне Србије', 'Central and West'],
           ['Регион Јужне и Источне Србије', 'South and East']]
for i in regions:
    income_hh.loc[income_hh['Region']==i[0], 'Region'] = i[1]
    spending_hh.loc[spending_hh['Region']==i[0], 'Region'] = i[1]
    
datasets.append([income_hh, 'income_hh'])
datasets.append([spending_hh, 'spending_hh'])

## 2. Foreign Trades

In [12]:
yearly_net_trade = pd.read_csv('02yearly_net_trade.csv', sep=';')
monthly_exports = pd.read_csv('02monthly_exports.csv', sep=';')
monthly_imports = pd.read_csv('02monthly_imports.csv', sep=';')
sector_exports = pd.read_csv('02sector_exports.csv', sep=';')
sector_imports = pd.read_csv('02sector_imports.csv', sep=';')

#Renaming and dropping columns
yearly_net_trade = yearly_net_trade.drop(['idindikator', 'IDVrPod', 'mes'], axis=1)
yearly_net_trade = yearly_net_trade.rename(columns={'nVrPod':'Action', 'god':'Year',
                                                    'vrednost':'Amount'})
#Translating
actions = [['Извоз', 'Exports(tUSD)'], ['Увоз', 'Imports(tUSD)'], ['Салдо', 'Net Exports(tUSD)']]
for i in actions:
    yearly_net_trade.loc[yearly_net_trade['Action']==i[0], 'Action'] = i[1]
    
#Unmelting
yearly_net_trade = yearly_net_trade.pivot_table(index='Year', columns='Action')
yearly_net_trade.columns = yearly_net_trade.columns.droplevel().rename(None)
yearly_net_trade = yearly_net_trade.reset_index()
    

#Renaming and dropping columns
monthly_exports = monthly_exports.drop(['idindikator', 'IDVrPod', 'nVrPod'], axis=1)
monthly_exports = monthly_exports.rename(columns={'god':'Year', 'mes':'Month',
                                                  'vrednost':'Amount(tUSD)'})
monthly_imports = monthly_imports.drop(['idindikator', 'IDVrPod', 'nVrPod'], axis=1)
monthly_imports = monthly_imports.rename(columns={'god':'Year', 'mes':'Month',
                                                  'vrednost':'Amount(tUSD)'})
#Dropping redundant rows
monthly_exports = monthly_exports.iloc[2::3]
monthly_imports = monthly_imports.iloc[2::3]

#Merging monthly_exports and monthly_imports and adding Net Exports
monthly_net_trade = pd.concat([monthly_exports, monthly_imports['Amount(tUSD)']], axis=1)
monthly_net_trade.columns = ['Month', 'Year', 'Exports(tUSD)', 'Imports(tUSD)']
monthly_net_trade['NetExports(tUSD)'] = monthly_net_trade['Exports(tUSD)'] - monthly_net_trade['Imports(tUSD)']
monthly_net_trade = monthly_net_trade.loc[monthly_net_trade['Month']!=0].reset_index(drop=True)
monthly_net_trade = monthly_net_trade.loc[monthly_net_trade['Year']!=2020].reset_index(drop=True)


#Renaming and dropping columns
sector_exports = sector_exports.drop(['idindikator', 'mes', 'nDrzave', 'IDNSST', 'IDVrPod'],
                                     axis=1)
sector_exports = sector_exports.rename(columns={'god':'Year', 'IDDrzave':'CountryId',
                                                'nNSST':'Sector', 'nVrPod':'Currency',
                                                'vrednost':'Amount(tUSD)'})
sector_imports = sector_imports.drop(['idindikator', 'mes', 'nDrzave', 'IDNSST', 'IDVrPod'],
                                     axis=1)
sector_imports = sector_imports.rename(columns={'god':'Year', 'IDDrzave':'CountryId',
                                                'nNSST':'Sector', 'nVrPod':'Currency',
                                                'vrednost':'Amount(tUSD)'})
#Sorting rows
sector_imports = sector_imports.loc[sector_imports['Currency']=='Вредност у хиљадама УСД']
sector_exports = sector_exports.loc[sector_exports['Currency']=='Вредност у хиљадама УСД']
sector_imports = sector_imports.drop('Currency', axis=1).reset_index(drop=True)
sector_exports = sector_exports.drop('Currency', axis=1).reset_index(drop=True)

#Merging
net_sector = pd.merge(sector_exports, sector_imports, on=['Year', 'CountryId', 'Sector'])
net_sector = net_sector.rename(columns={'Amount(tUSD)_x':'Exports(tUSD)',
                                        'Amount(tUSD)_y':'Imports(tUSD)'})
net_sector['NetExports(tUSD)'] = net_sector['Exports(tUSD)'] - net_sector['Imports(tUSD)']

aggregates = [['00', 'Total'], ['01', 'EU'], ['02', 'CEFTA']]
for i in aggregates:
    net_sector.loc[net_sector['CountryId']==i[0], 'CountryId'] = i[1]

#Translating
sectors = [['Храна и живе животиње', 'Food and Livestock'], 
           ['Пића и дуван', 'Drinks and Tobacco'],
           ['Сирове материје, нејестиве, осим горива', 'Ineadible Raw Materials Excluding Fuel'],
           ['Минерална горива, мазива и сродни производи', 'Mineral Fuels, Lubricants and Related Products'],
           ['Животињска и биљна уља, масти и воскови', 'Animal and Vegetable Oils, Fats and Waxes'],
           ['Хемијски и сл. производи, нигде непоменути', 'Chemical Products and Similar'],
           ['Израђени производи сврстани по материјалу', 'Manufactured Products Classified by Material'],
           ['Машине и транспортни уређаји', 'Machines and Transport Devices'], 
           ['Разни готови производи', 'Other Finished Products'],
           ['Производи непоменути у СМТК Рев. 4', 'Undeclared Products within SITC Rev. 4']]

for i in sectors:
    net_sector.loc[net_sector['Sector']==i[0], 'Sector'] = i[1]
    
datasets.append([yearly_net_trade, 'yearly_net_trade'])
datasets.append([monthly_net_trade, 'monthly_net_trade'])
datasets.append([net_sector, 'net_sector'])

## 3. CPI

In [4]:
cpi = pd.read_json('03cpi.json')

#Renaming and dropping columns and rows
cpi = cpi.loc[cpi['nVrPod']=='Базни индекси (2006 = 100)'].reset_index(drop=True)
cpi = cpi.drop(['idindikator', 'IDVrPod', 'nVrPod', 'IDTer', 'nTer', 'IDCOICOP'], axis=1)
cpi = cpi.rename(columns={'mes':'Month', 'god':'Year', 'nCOICOP':'ExpenditureCategory',
                 'vrednost':'CPI(2006=100)'})
cpi = cpi.loc[cpi['Year']!=2020].reset_index(drop=True)
cpi = cpi.drop_duplicates()

#Translating
categories = [['Укупно', 'Total'], ['Храна и безалкохолна пића', 'Food and Non-alcoholic Drinks'],
              ['Храна', 'Food'], ['Хлеб и житарице', 'Bread and Cereal'],
              ['Месо', 'Meat'], ['Риба', 'Fish'], ['Млеко, сир и јаја', 'Milk, Cheese and Eggs'], 
              ['Уља и масти', 'Oils and Fats'], ['Воће', 'Fruit'],
              ['Поврће', 'Vegetables'], ['Шећер, џем, мед и чоколада', 'Sugar, Jam, Honey and Chocolate'],
              ['Остали прехрамбени производи', 'Other Eatable Prducts'], 
              ['Безалкохолна пића', 'Non-alcoholic Drinks'], ['Кафа, чај и какао', 'Coffee, Tea and Cocoa'], 
              ['Минерална вода, безалкохолна пића и сокови', 'Mineral Water, Non-alcoholic Drinks and Juices'],
              ['Алкохолна пића, дуван и наркотици', 'Alcoholic Drinks, Tobacco and Narcotics'], 
              ['Алкохолна пића', 'Alcoholic Drinks'],
              ['Жестока алкохолна пића', 'Strong Alcoholic Drinks'], ['Вино', 'Wine'], ['Пиво', 'Beer'], 
              ['Дуван', 'Tobacco'], ['Одећа и обућа', 'Clothing and Footwear'],
              ['Одећа', 'Clothing'], ['Материјал за одећу', 'Clothing Material'],
              ['Одевни предмети', 'Garments'], ['Остали одевни предмети', 'Other Garments'], 
              ['Чишћење, поправка и шивење одеће', 'Clothing Maintenance'],
              ['Обућа', 'Footwear'], ['Поправка обуће', 'Footwear Maintenance'],
              ['Становање, вода, ел. енергија, гас и остала горива', 'Housing, Water, Energy, Gas and Other Fuels'],
              ['Стварне стамбене ренте', 'Real Rent'], ['Изнајмљивање стана', 'Rent'],
              ['Oдржавање и поправка станова', 'Apartment Maintenance'],
              ['Материјал за одржавање и поправку станова', 'Materials for Apartment Maintenance'],
              ['Услуге за одржавање и поправку станова', 'Apartment Maintenance Services'],
              ['Снабдевање водом и остале стамбене услуге', 'Waterworks and Other Housing Services'], 
              ['Снабдевање водом', 'Water Supply'],
              ['Одношење смећа', 'Garbage Collection'], 
              ['Одвoђење отпадне воде', 'Waste Water Drainage'],
              ['Електрична енергија, гас и остала горива', 'Energy, Gas and Other Fuels'],
              ['Електрична енергија за домаћинство', 'Hosehold Electric Energy'], 
              ['Гас', 'Gas'], ['Течна горива', 'Liquid Fuels'],
              ['Чврста горива', 'Solid Fuels'], ['Даљинско грејање', 'Distric Heating'],
              ['Опрема за стан и текуће одржавање', 'Household Equipment and Maintenance'],
              ['Намештај, теписи и остале подне простирке', 'Furniture, Carpets and Other Rugs'],
              ['Намештај, расвета и декоративна опрема за стан', 'Furniture, Lightning and Decorative Ornaments'],
              ['Теписи и остале подне простирке', 'Carpets and Other Floor Rugs'], 
              ['Текстил за домаћинство', 'Textile for Household Use'],
              ['Апарати за домаћинство', 'Household Appliances'], ['Велики кућни апарати', 'Big Household Appliances'],
              ['Мали електрични апарати', 'Small Electrical Appliances'], 
              ['Поправка апарата за домаћинство', 'Household Appliances Maintenance'],
              ['Посуђе и остали прибор за јело', 'Dishes and Cutlery'],
              ['Алат и остала опрема за кућу и врт', 'Tools and Other Equipment for House and Garden'],
              ['Већи алат и опрема за кућу и врт', 'Bigger House and Garden Tools and Equipment'],
              ['Мали алат и разноврсни прибор', 'Small Tools and Accessories'],
              ['Средства и услуге за текуће одржавање стана', 'Funds and Services for Ongoing Maintenance of Dwelling'],
              ['Средства за одржавање стана', 'Dwelling Maintenance Funds'], ['Услуге за текуће одржавање стана', 'Dwelling Maintenance Services'],
              ['Здравље', 'Health'], ['Лекови и медицински уређаји и опрема', 'Medication and Medical Equipment'], ['Лекови', 'Medication'],
              ['Остали медицински производи', 'Other Medical Products'], ['Здравствене неболничке услуге', 'Non-hospital Health Services'],
              ['Медицинске услуге', 'Medical Services'], ['Стоматолошке услуге', 'Dentist Services'],
              ['Пратеће медицинске услуге', 'Accompanying Medical Services'], ['Транспорт', 'Transport'],
              ['Набавка возила', 'Procurement of Vehicles'],
              ['Аутомобили', 'Cars'], ['Бицикли', 'Bycicles'], ['Коришћење и одржавање возила', 'Vehicles Use and Maintenance'],
              ['Резервни делови', 'Reserve Parts'], ['Горива и мазива за путничка возила', 'Fuels and Lubricants for Passenger Vehicles'],
              ['Одржавање и поправка путничких возила', 'Maintenance of Passenger Vehicles'],
              ['Остале услуге у вези са употребом возила', 'Other Vehicle Related Services'], ['Транспортне услуге', 'Transport Services'],
              ['Превоз путника железницом', 'Rail Passenger Transport'], ['Превоз путника друмом', 'Road Passenger Transport'],
              ['Превоз путника авионом', 'Plane Passenger Transport'], ['Комуникације', 'Communication'], ['Поштанске услуге', 'Mail Services'],
              ['Телефонска опрема', 'Telephone Equipment'], ['Телефонске и остале услуге', 'Telephone and Other Equipment'],
              ['Телефонске услуге', 'Telephone Services'], ['Рекреација и култура', 'Culture and Recreation'],
              ['Аудио-визуелна, фотографска и рачунарска опрема', 'Audio-Video, Photographic and Computer Equipment'],
              ['Опрема за пријем, снимање и репродукцију звука и слике', 'Sound and Picture Reciving, Capturing and Reproduction Equipment'],
              ['Фотографска и филмска опрема', 'Photographic and Film Equipment'], ['Рачунарска опрема', 'Computer Equipment'],
              ['Медији за снимање слике и звука', 'Sound and Picture Capturing Medium'],
              ['Поправка аудио-визуелне, фотографске и рачунарске опреме', 'Audio-Visual, Photographic and Computer Equipment Maintenance'],
              ['Већа трајна добра за рекреацију и културу', 'Grearer Lasting Goods for Culture and Recreation'], ['Музички инструменти', 'Musical Instruments'],
              ['Остала опрема за рекреацију, врт и кућни љубимци', 'Other Recreational, Garden and Pet-related Equipment'],
              ['Играчке, игре и хоби', 'Toys, Games and Hobbies'], ['Баште, саднице и цвеће', 'Garden, Seedling and Flower'],
              ['Кућни љубимци, средства и услуге у вези са  кућним љубимцима', 'Pets and Pet-related Means and Services'],
              ['Ветеринарске и друге услуге за животиње', 'Vet and Other Pet Services'],
              ['Рекреација и култура – услуге', 'Cultural and Recreational Services'], ['Рекреација – услуге', 'Recreational Services'],
              ['Култура – услуге', 'Cultural Services'], ['Новине, књиге и канцеларијски прибор', 'Newspaper, Books and Office Equipment'],
              ['Књиге', 'Books'], ['Новине и часописи', 'Newspapers and Magazines'], ['Канцеларијски материјал', 'Office Equipments'],
              ['Образовање', 'Education'], ['Средњошколско образовање', 'Highschool Education'],
              ['Више и високо образовање', 'Higher Education'], ['Остали видови образовања', 'Other Types of Education'],
              ['Ресторани и хотели', 'Restaurants and Hotels'], ['Ресторани, кафеи и кантине', 'Restaurants, Cafes and Canteens'],
              ['Ресторани и кафеи', 'Restaurants and Cafes'], ['Услуге смештаја', 'Accommodation Services'], ['Смештај', 'Accommodation'],
              ['Остали лични предмети и остале услуге', 'Other Personal Items and Other Services'], ['Лична нега', 'Personal Care'],
              ['Услуге у фризерским и козметичким салонима', 'Cosmetic and Hairdresser Services'],
              ['Електрични апарати за личну негу', 'Personal Care Electrical Equipment'],
              ['Остaли предмети за личну негу', 'Other Personal Care Items'], ['Лични предмети', 'Personal Items'],
              ['Накит и сатови', 'Jewelry and Watches'], ['Остали лични предмети', 'Other Personal Items'], ['Социјална заштита', 'Social Security'],
              ['Осигурање', 'Insurance'], ['Осигурање стана', 'Dwelling Insurance'], ['Осигурање транспортних возила', 'Vehicle Insurance'],
              ['Финансијске услуге', 'Financial Services'],
              ['Финансијске услуге на другом месту непоменуте', 'Other Financial Services'], ['Остале услуге', 'Other Services'],
              ['Терапеутски производи и опрема', 'Therapeutic Products and Equipment'], ['Опрема за спорт и камповање', 'Camping and Sport Equipment'],
              ['Поправка намештаја, расвете и подних простирки', 'Repair of Furniture, Lighting and Floor Coverings'],
              ['Пакет-аранжмани', 'Package Deals']]
for i in categories:
    cpi.loc[cpi['ExpenditureCategory']==i[0], 'ExpenditureCategory'] = i[1]
    
datasets.append([cpi, 'cpi'])

## 4. Industry

In [5]:
#Industry by Sector
industry_sector = pd.read_csv('04industry_traffic_index15.csv', sep=';')

#Renaming and dropping columns and rows
industry_sector = industry_sector.drop(['idindikator', 'nTer', 'IDTer', 'IDVrNamene'], axis=1)
industry_sector = industry_sector.rename(columns={'mes':'Month', 'god':'Year',
                'nVrNamene':'SectorPurpose', 'vrednost':'ProductionIndex(2015=100)'})
industry_sector = industry_sector.loc[industry_sector['Year']!=2020].reset_index(drop=True)
industry_sector = industry_sector.drop_duplicates()

#Translating
industrial_sectors = [['Укупно', 'Total'], ['Енергија', 'Energy'], 
                      ['Интермедијарни производи', 'Intermediate Products'],
                      ['Капитални производи', 'Capital Products'], 
                      ['Трајни производи за широку потрошњу', 'Durable Consumer Goods'],
                      ['Нетрајни производи за широку потрошњу', 'Temporary Consumer Goods']]
for i in industrial_sectors:
    industry_sector.loc[industry_sector['SectorPurpose']==i[0], 'SectorPurpose'] = i[1]

#Splitting Data into Monthly and Yearly
yearly_industry_sector = industry_sector.loc[industry_sector['Month']==0]
yearly_industry_sector = yearly_industry_sector.drop('Month', axis=1).reset_index(drop=True)

monthly_industry_sector = industry_sector.loc[industry_sector['Month']!=0]
monthly_industry_sector = monthly_industry_sector.reset_index(drop=True)


#Industry by Activity
industry_activity = pd.read_csv('04industry_activity.csv', sep=';')

#Renaming and dropping columns and rows
industry_activity = industry_activity.drop(['idindikator', 'nTer', 'IDTer', 'IDKD08'], axis=1)
industry_activity = industry_activity.rename(columns={'mes':'Month', 'god':'Year',
                'nkd08':'Activity', 'vrednost':'ProductionIndex'})
industry_activity = industry_activity.loc[industry_activity['Year']!=2020].reset_index(drop=True)
industry_activity = industry_activity.drop_duplicates()

#Translating
industrial_activities = [['Укупно', 'Total'], ['Експлоатација угља', 'Coal Exploitation'],
                         ['Експлоатација сирове нафте и природног гаса', 'Oil and Natural Gas Exploitation'],
                         ['Експлоатација руда метала', 'Metal Ore Exploitation'], 
                         ['Остало рударство', 'Other Types of Mining'],
                         ['Производња прехрамбених производа', 'Food Products Production'], 
                         ['Производња пића', 'Drink Production'],
                         ['Производња дуванских производа', 'Tobacoo Products Production'], 
                         ['Производња текстила', 'Textile Production'],
                         ['Производња одевних предмета', 'Clothing Production'],
                         ['Производња коже и предмета од коже', 'Leather and Leater Item Production'],
                         ['Прерада дрвета и производи од дрвета, плуте, сламе и прућа, осим намештаја', 'Wood and Wood, Cork, Straw and Rod - Related Products Production, Except Furniture'],
                         ['Производња папира и производа од папира', 'Paper and Paper Products Production'],
                         ['Штампање и умножавање аудио и видео записа', 'Printing and Duplication of Audio and Video Material'],
                         ['Производња кокса и деривата нафте', 'Coke and Refined Petroleum Production'],
                         ['Производња хемикалија и хемијских производа', 'Chemicals and Chemical Products Production'],
                         ['Производња основних фармацеутских производа и препарата', 'Basic Pharmaceutical Products and Remedies Production'],
                         ['Производња производа од гуме и пластике', 'Rubber and Plastic Production'],
                         ['Производња производа од осталих неметалних минерала', 'Manufacture of Other Non-metallic Minerals Products'],
                         ['Производња основних метала', 'Base Metals Production'],
                         ['Производња металних производа, осим машина и уређаја', 'Fabricated Metal Products Production, Except Machinery and Equipment'],
                         ['Производња рачунара, електронских и оптичких производа', 'Computer, Electronic and Optical Products Production'],
                         ['Производња електричне опреме', 'Electrical Equipment Production'],
                         ['Производња непоменутих машина и непоменуте опреме', 'Unmentioned Machinery and Equipment Production'],
                         ['Производња моторних возила, приколица и полуприколица', 'Motor Vehicles, Trailers and Semi-trailer Production'],
                         ['Производња осталих саобраћајних средстава', 'Other Transport Equipment Production'],
                         ['Производња намештаја', 'Furniture Production'], ['Остале прерађивачке делатности', 'Other Manufacturing'],
                         ['Поправка и монтажа машина и опреме', 'Repair and Installation of Machinery Equipment'],
                         ['Снабдевање електричном енергијом, гасом, паром и климатизација', 'Electricity, Gas, Steam and Air Conditioning Supply'],
                         ['B - Рударство', 'B - Mining'], ['C - Прерађивачка индустрија', 'C - Manufacturing Industry'],
                         ['D - Снабдевање електричном енергијом, гасом, паром и климатизација', 'D - Electricity, Gas, Steam and Air Conditioning Supply']]
for i in industrial_activities:
    industry_activity.loc[industry_activity['Activity']==i[0], 'Activity'] = i[1]

#Splitting Data into Monthly and Yearly
yearly_industry_activity = industry_activity.loc[industry_activity['Month']==0]
yearly_industry_activity = yearly_industry_activity.drop('Month', axis=1).reset_index(drop=True)

monthly_industry_activity = industry_activity.loc[industry_activity['Month']!=0]
monthly_industry_activity = monthly_industry_activity.reset_index(drop=True)

datasets.append([yearly_industry_sector, 'yearly_industry_sector'])
datasets.append([monthly_industry_sector, 'monthly_industry_sector'])
datasets.append([yearly_industry_activity, 'yearly_industry_activity'])
datasets.append([monthly_industry_activity, 'monthly_industry_activity'])

## 5. National accounts

In [2]:
#GDP and Other GDP-related Stats
gdp = pd.read_csv('05gdp.csv', sep=';')

#Renaming and dropping columns and rows
gdp = gdp.drop(['idindikator', 'IDTer', 'nTer', 'IDStavkeBDP', 'IDVrPod', 'mes'], axis=1)
gdp = gdp.rename(columns={'nStavkeBDP':'EditMe1', 'nVrPod':'EditMe2', 'god':'Year',
                          'vrednost':'EditMe3'})
gdp = gdp.loc[gdp['EditMe2']!='Вредност, сталне цене (цене претходне године), мил. РСД']
gdp = gdp.reset_index(drop=True)

#Translating
editme2 = [['Вредност, текуће цене, мил. РСД', 'NominalValue(mRSD)'], 
           ['Учешће у БДП, %', '%NominalGDP'],
           ['Вредност, уланчане мере обима, референтна 2010. година, мил. РСД', 'RealValue(mRSD,2010=100)'],
           ['Стопе реалног раста, претходна година = 100, %', 'RealGrowthRatePct(LastYear=100)']]
for i in editme2:
    gdp.loc[gdp['EditMe2']==i[0], 'EditMe2'] = i[1]
    
editme1 = [['Бруто додата вредност (БДВ)', 'GrossValueAdded'],
           ['Порези на производе', 'ProductTax'],
           ['Субвенције на производе', 'ProductSubventions'], 
           ['Бруто домаћи производ (БДП)', 'GDP']]
for i in editme1:
    gdp.loc[gdp['EditMe1']==i[0], 'EditMe1'] = i[1]

#Pivoting
gdp = gdp.pivot_table(index=['EditMe1', 'Year'], columns='EditMe2')
gdp.columns = gdp.columns.droplevel().rename(None)
gdp.index.names = ['Account', 'Year']
gdp = gdp.reset_index()


#Added Value by Actions by Q
qav_actions = pd.read_csv('05quartal_added_value_by_actions.csv', sep=';')

#Renaming and dropping columns and rows
qav_actions = qav_actions.drop(['idindikator', 'IDTer', 'nTer', 'IDKD08', 'IDVrPod'], axis=1)
qav_actions = qav_actions.rename(columns={'nkd08':'Action', 'nVrPod':'EditMe1', 'mes':'Q', 'god':'Year',
                          'vrednost':'EditMe2'})
qav_actions = qav_actions.loc[qav_actions['EditMe1']!='Вредност, сталне цене (цене претходне године), мил.РСД']
qav_actions = qav_actions.reset_index(drop=True)

#Translating
editme1 = [['Вредност, текуће цене, мил.РСД', 'NominalValue(mRSD)'], 
           ['Учешће у БДП, %', '%NominalGDP'],
           ['Вредност, уланчане мере обима, референтна 2010. година, мил.РСД', 'RealValue(mRSD,2010=100)'],
           ['Стопе реалног раста, уланчане мере обима (исти квартал претходне године = 100), %', 'RealGrowthRatePct(LastYear=100)']]
for i in editme1:
    qav_actions.loc[qav_actions['EditMe1']==i[0], 'EditMe1'] = i[1]
    
actions = [['Укупно', 'Total'], ['A - Пољопривреда, шумарство и рибарство', 'A - Agriculture, Forestry and Fishing'],
           ['B-E - Прерађивачка индустрија, рударство и остала индустрија', 'B-E - Manufacturing, Mining and Other Industries'],
           ['F - Грађевинарство', 'F - Construction'],
           ['G-I - Трговина на велико и мало, саобраћај и складиштење и услуге смештаја и исхране', 'Wholesale and Retail trade, Transport and Storage, Lodging and Catering Services'],
           ['J - Информисање и комуникације', 'J - Informing and Communication'],
           ['K - Финансијске делатности и делатност осигурања', 'K - Financial and Insurance Activities'],
           ['L - Пoсловање некретнинама', 'L - Real Estate Business'],
           ['M-N - Стручне, научне, техничке, административне и друге помоћне активности', 'M-N - Professional, Scientific, Technical, Administrative and Other Ancillary Activities'],
           ['O-Q - Државна управа, одбрана, образовање и делатности у оквиру здравствене и социјалне заштите', 'O-Q - Public Administration, Defense, Education and Health, Social Care'],
           ['R-T - Остале услужне делатности', 'R-T - Other Service Activities']]
for i in actions:
    qav_actions.loc[qav_actions['Action']==i[0], 'Action'] = i[1]
    
qs = [['K1', 'Q1'], ['K2', 'Q2'], ['K3', 'Q3'], ['K4', 'Q4']]
for i in qs:
    qav_actions.loc[qav_actions['Q']==i[0], 'Q'] = i[1]

#Pivoting
qav_actions = qav_actions.pivot_table(index=['Year', 'Q', 'Action'], columns='EditMe1')
qav_actions.columns = qav_actions.columns.droplevel().rename(None)
qav_actions = qav_actions.reset_index()


#Government Investitions
gov_investitions = pd.read_csv('05investitions.csv', sep=';')

#Renaming and dropping columns and rows
gov_investitions = gov_investitions.drop(['idindikator', 'IDTer', 'nTer', 'IDTehnickaStruktura',
                                'mes'], axis=1)
gov_investitions = gov_investitions.rename(columns={'nTehnickaStruktura':'Investition', 
                                          'god':'Year', 'vrednost':'Amount(mRSD)'})
#Translating
investitions = [['Укупно', 'Total'],
               ['Зграде и остале грађевине', 'Buildings and Other Construction Work'], 
               ['Стамбене зграде', 'Apartment Buildings'],
               ['Нестамбене зграде и остале грађевине', 'Non-apartment Buildings and Other Construction Work'],
               ['Машине и опрема (+ војна опрема)', 'Machinery, Euipment and Military Equipment'],
               ['Култивисани биолошки ресурси', 'Cultivated Biological Resources'],
               ['Интелектуална својина', 'Intellectual Property']]
for i in investitions:
    gov_investitions.loc[gov_investitions['Investition']==i[0], 'Investition'] = i[1]
    
    
#Government Taxes and Social Contributions
gov_tax = pd.read_csv('05tax_and_social_contributions.csv', sep=';')

gov_tax = gov_tax.sort_values(by='god')
gov_tax = gov_tax.reset_index(drop=True)

#Renaming and dropping columns and rows
gov_tax = gov_tax.drop(['idindikator', 'IDTer', 'nTer', 'IDTransakcije', 'mes'], axis=1)     
gov_tax = gov_tax.rename(columns={'nTransakcije':'TransactionType', 'god':'Year', 
                                          'vrednost':'Amount(mRSD)'})
#Translating
transactions = [['УКУПНO', 'Total'], ['ПОРЕСКА ПРИМАЊА', 'Tax Receipts'], 
                ['ПОРЕЗИ НА ПРОИЗВОДЊУ И УВОЗ', 'Production and Import Taxes'],
                ['Порези на производе', 'Product Taxes'], 
                ['Порез на додату вредност', 'Value Added Taxes'],
                ['Порези и дажбине на увоз, осим ПДВ', 'Import Taxes and Duties, Except VAT'],
                ['Додаци на пензијско осигурање домаћинстава', 'Household Pension Supplement'],
                ['Додаци на осигурање домаћинстава, осим пензијског осигурања', 'Household insurance supplements other than pension insurance'],
                ['КАПИТАЛНИ ПОРЕЗИ', 'Capital Taxes'], 
                ['Порези на капиталне трансфере', 'Capital Transfer Taxes'],
                ['Капиталне дажбине', 'Capital Duties'], 
                ['Остали непоменути капитални порези', 'Other Unmentioned Capital Taxes'],
                ['Импутирани доприноси за пензијско осигурање на терет послодавца', 'Imputed Retirement Contributions Carried by Employer'],
                ['Импутирани доприноси за осигурање на терет послодавца, осим пензијског осигурања', 'Imputed Contributions for Insurance at Expense of Employer Other Than Pension Insurance'],
                ['Стварни доприноси домаћинстава за социјално осигурање', 'Real Contributions from Households for Social Security'],
                ['Стварни доприноси домаћинстава за пензијско осигурање', 'Actual Pension Contributions by Households'],
                ['Стварни доприноси домаћинстава за осигурање, осим пензијског осигурања', 'Real Contributions from Households for Insurance Other Than Pension Insurance'],
                ['Додаци на социјално осигурање домаћинстава', 'Household Social Security Supplements'],
                ['Остали непоменути текући порези', 'Other Unmentioned Current Taxes'], ['НЕТО СОЦИЈАЛНИ ДОПРИНОСИ', 'Net Social Contributions'],
                ['Стварни доприноси за социјално осигурање на терет послодавца', 'Real Employer Social Security Contributions'],
                ['Стварни доприноси  за пензијско осигурање на терет послодавца', 'Real Employer Retirement Contributions'],
                ['Стварни доприноси за осигурање на терет послодавца, осим пензијског осигурања', 'Real Employer Insurance Contributions, Other Than Pension Insurance'],
                ['Импутирани социјални доприноси на терет послодавца', 'Imputed Social Contributions at Expense of Employer'],
                ['Остали текући порези', 'Other Current Taxes'], ['Текући порези на капитал', 'Current Capital Taxes'],
                ['Порези по становнику', 'Per Capita Taxes'], ['Порези на издатке за потрошњу', 'Consumption Expenditure Taxes'],
                ['Плаћања домаћинстава за дозволе', 'Non-Corporate Permit Payments'],
                ['Порези на међународне трансакције', 'International Transactions Taxes'],
                ['Порези на власнички добитак предузећа', 'Enterprise Ownership Taxes'],
                ['Остали порези на власнички добитак', 'Other Ownership Taxes'],
                ['Порези на добитке од игара на срећу или коцкања', 'Gambling Gain Taxes'],
                ['Остали непоменути порези на доходак', 'Other Unmentioned Income Taxes'],
                ['Порези на доходак појединца или домаћинства, укључујући власнички добитак', 'Individual or Household Income Tax, Includion Ownership Profit'],
                ['Порези на приход или профит предузећа, укључујући власнички добитак', 'Corporate Income or Profit Tax, Including Ownership Profit'],
                ['ТЕКУЋИ ПОРЕЗИ НА ДОХОДАК, БОГАТСТВО ИТД.', 'Current Income Taxes, Wealth, Etc.'],
                ['Порези на доходак', 'Income Taxes'],
                ['Порези на доходак појединца или домаћинства, осим власничког добитка', 'Individual or Household Income Tax Other Than Ownership Profit'],
                ['Порези на  приход или профит предузећа, осим власничког добитка', 'Corporate Income or Profit Taxes Other Than Ownership Profit'],
                ['Порези на власнички добитак', 'Ownership Profit Taxes'],
                ['Порези на власнички добитак појединца или домаћинства', 'Individual or Household Equity Taxes'],
                ['Порез на фонд зарада', 'Wage Fund Taxes'], ['Пословне и професионалне дозволе', 'Business and Professional Licenses'],
                ['Порези на загађење', 'Pollution Taxes'], ['Под-компензације ПДВ (паушални систем)', 'Sub-compensation of Flat-rate VAT'],
                ['Остали непоменути порези на производњу', 'Other Unmentioned Production Taxes'],
                ['Профит фискалних монопола', 'Fiscal Monopoly Profit'],
                ['Извозне дажбине и новчане компензације на извоз', 'Export Duties and Refunds'],
                ['Остали непоменути порези на производе', 'Other Unmentioned Product Taxes'],
                ['Остали порези на производњу', 'Other Production Taxes'],
                ['Порези на земљиште, зграде и друге објекте', 'Land, Building and Other Real Estate Taxes'],
                ['Порези на употребу основних фондова', 'Use of Fixed Assets Taxes'],
                ['Порези на регистрацију возила', 'Vehicle Registration Taxes'], 
                ['Порези на забаву', 'Entertainment Taxes'],
                ['Порези на лутрију, игре на срећу и клађење', 'Lottery, Chance Games and Gambling Tax'],
                ['Порез на премије осигурања', 'Insurance Premiums Taxes'], 
                ['Остали порези на посебне услуге', 'Other Special Servies Taxes'],
                ['Општи порези на промет или продају', 'General Sales Tax'], ['Порез на посебне услуге', 'Special Services Tax'],
                ['Профит монопола на увоз', 'Import Monopoly Profit'],
                ['Порези на производе, осим ПДВ и порезе на увоз', 'Product Taxes Other than VAT and Import Taxes'],
                ['Акцизе и порези на потрошњу', 'Excise and Consumption Taxes'], ['Таксене марке', 'Tax Stamps'],
                ['Порези на финансијске и капиталне трансакције', 'Capital and Financial Transactions Taxes'],
                ['Увозне дажбине', 'Import Duties'], ['Порези на увоз, осим ПДВ и увозних дажбина', 'Import Taxes, Except VAT and Import Duties'],
                ['Дажбине на увоз пољопривредних производа', 'Agricultural Products Import Duties'],
                ['Новчане компензације на увоз', 'Cash Import Compensation'], ['Акцизе', 'Excise Duties'], ['Општи порез на промет', 'General Sales Tax']]
for i in transactions:
    gov_tax.loc[gov_tax['TransactionType']==i[0], 'TransactionType'] = i[1]

#Imputing Percent of Total Taxation of Different Tax Groups
tax_datas = []
for year in gov_tax['Year'].unique():
    curr_data = gov_tax.loc[gov_tax['Year']==year]
    total = curr_data.loc[curr_data['TransactionType']=='Total', 'Amount(mRSD)'].reset_index(drop=True).at[0]
    curr_data['PctTotal'] = round(curr_data['Amount(mRSD)'] / total, 4)
    tax_datas.append(curr_data)
gov_tax = pd.concat(tax_datas)

datasets.append([gdp, 'gdp'])
datasets.append([qav_actions, 'qav_actions'])
datasets.append([gov_investitions, 'gov_investitions'])
datasets.append([gov_tax, 'gov_tax'])

C:\Users\vucin\Anaconda3\lib\site-packages\ipykernel_launcher.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 6. Population

In [7]:
#Population by Age, Region and Gender
population = pd.read_csv('06population.csv', sep=';')

#Translating
regions = [['РЕПУБЛИКА СРБИЈА', 'State'], ['Београдски регион', 'Belgrade'], 
           ['Регион Војводине', 'Voyvodina/North'], 
           ['Регион Шумадије и Западне Србије', 'Central and West'],
           ['Регион Јужне и Источне Србије', 'South and East']]
for i in regions:
    population.loc[population['nTer']==i[0], 'nTer'] = i[1]
    
genders = [['Укупно', 'Total'], ['Женско', 'F'], ['Мушко', 'M']]
for i in genders:
    population.loc[population['nPol']==i[0], 'nPol'] = i[1]

age_groups = [['Укупно', '-1'], ['85 и више', '85']]
for i in age_groups:
    population.loc[population['nStarost']==i[0], 'nStarost'] = i[1]
    
#Renaming and dropping columns and rows
population = population.loc[(population['nTer']!='СРБИЈА – СЕВЕР')&(population['nTer']!='СРБИЈА – ЈУГ')]
population = population.reset_index(drop=True)

population = population.drop(['idindikator', 'mes', 'IDTer', 'IDPol', 'IDStarost'], axis=1)
population = population.rename(columns={'god':'Year', 'nTer':'Region', 'nPol':'Gender',
                                        'nStarost':'AgeGroup', 'vrednost':'Amount'})

population['AgeGroup'] = population['AgeGroup'].astype('int')


#Life Expectancy
life_expectancy = pd.read_csv('06life_expectancy.csv', sep=';')

#Translating

for i in regions:
    life_expectancy.loc[life_expectancy['nTer']==i[0], 'nTer'] = i[1]
    
for i in genders:
    life_expectancy.loc[life_expectancy['nPol']==i[0], 'nPol'] = i[1]

#Renaming and dropping columns and rows
life_expectancy = life_expectancy.drop(['idindikator', 'mes', 'IDTer', 'IDPol', 'IDStarGrupa'], axis=1)
life_expectancy = life_expectancy.rename(columns={'god':'Year', 'nTer':'Region', 'nPol':'Gender',
                                        'nStarGrupa':'AgeGroup', 'vrednost':'Amount'})

#Sorting Useful Rows
life_expectancy = life_expectancy.loc[(life_expectancy['Region']!='СРБИЈА – СЕВЕР')&(life_expectancy['Region']!='СРБИЈА – ЈУГ')]
life_expectancy = life_expectancy.loc[life_expectancy['AgeGroup']=='0']
life_expectancy = life_expectancy.drop('AgeGroup', axis=1)
life_expectancy = life_expectancy.sort_values(['Year', 'Region', 'Gender'])
life_expectancy = life_expectancy.reset_index(drop=True)


#Natural Increase
natural_increase = pd.read_csv('06natural_increase.csv', sep=';')

#Translating
for i in regions:
    natural_increase.loc[natural_increase['nTer']==i[0], 'nTer'] = i[1]
    
#Dropping Smaller Regions
smaller_regions = [i[1] for i in regions]
natural_increase = natural_increase.loc[natural_increase['nTer'].isin(smaller_regions)]
    
#Renaming and dropping columns
natural_increase = natural_increase.drop(['idindikator', 'mes', 'IDVrPod', 'IDTer'], axis=1)
natural_increase = natural_increase.rename(columns={'god':'Year', 'nTer':'Region', 'nVrPod':'EditMe1', 'vrednost':'Amount'})

#Debug Regions
#dummie = natural_increase.loc[natural_increase['Year']==2011]
#dummie = dummie.loc[dummie['EditMe1']=='Број становника средином године']
#dummie['Amount'][0]-dummie['Amount'][1:].sum()

#Dropping and Sorting Rows
natural_increase = natural_increase.loc[natural_increase['EditMe1']=='Природни прираштај']
natural_increase = natural_increase.drop('EditMe1', axis=1)
natural_increase = natural_increase.sort_values(['Year', 'Region']).reset_index(drop=True)


#Net Internal Imigrations
internal_migrations = pd.read_csv('06internal_migrations.csv', sep=';')

#Translating
for i in genders:
    internal_migrations.loc[internal_migrations['nPol']==i[0], 'nPol'] = i[1]
    
for i in regions:
   internal_migrations.loc[internal_migrations['nTer']==i[0], 'nTer'] = i[1]

#Dropping Smaller Regions
internal_migrations = internal_migrations.loc[internal_migrations['nTer'].isin(smaller_regions)]
    
#Renaming and dropping columns
internal_migrations = internal_migrations.drop(['idindikator', 'mes', 'IDVrPod', 'IDTer', 'IDStarGrupa', 'IDPol'], axis=1)
internal_migrations = internal_migrations.rename(columns={'god':'Year', 'nTer':'Region', 'nVrPod':'EditMe1', 'vrednost':'Amount', 'nPol':'Gender', 'nStarGrupa':'AgeGroup'})

#Rows
internal_migrations = internal_migrations.loc[internal_migrations['EditMe1']=='Миграциони салдо']
internal_migrations = internal_migrations.loc[internal_migrations['AgeGroup']=='Укупно']
internal_migrations = internal_migrations.drop(['EditMe1', 'AgeGroup'], axis=1)

internal_migrations = internal_migrations.reset_index(drop=True)

datasets.append([population, 'population'])
datasets.append([life_expectancy, 'life_expectancy'])
datasets.append([natural_increase, 'natural_increase'])
datasets.append([internal_migrations, 'internal_migrations'])

## 7. Energetics

In [8]:
energetics = pd.read_csv('07energetics.csv', sep=';')

#Dropping Redundant Rows
useful_rows = ['Примарна производња енергије', 'Увоз', 'Извоз', 'Салдо залиха',
               np.nan, 'РЕПУБЛИКА СРБИЈА', 'Укупно расположива енергија',
               'Статистичка разлика', 'Утрошак за производњу енергије',
               'Производња енергије трансформацијом', 'Размена', 'Размењени производи', 
               'Интерна размена производа', 'Враћено из петрохемије',
               'Сопствена потрошња у енергетском сектору', 'Губици',
               'Енергија расположива за финалну потрошњу',
               'Финална потрошња за неенергетске сврхе',]
energetics = energetics.loc[energetics['nTokovi'].isin(useful_rows)]

#Sources to Drop Check
#for i in energetics['EnergySources'].unique():
#    curr_data = energetics.loc[energetics['EnergySources']==i]
#    if curr_data['Amount'].sum()==0:
#        print('Energy Source:\n', i, curr_data['Amount'].sum(), '\n')
sources_to_drop = ['Соларна фотонапонска енергија, GWh', 'Енергија ветра, GWh',
                   'Хидро енергија, GWh']
energetics = energetics.loc[~energetics['nEnergenti'].isin(sources_to_drop)]

energetics = energetics.loc[energetics['nTer']=='РЕПУБЛИКА СРБИЈА']

#Renaming and dropping columns
energetics = energetics.drop(['idindikator', 'IDTokovi', 'IDTer', 'nTer', 'mes', 'IDEnergenti'], axis=1)
energetics = energetics.rename(columns={'nTokovi':'Use', 'god':'Year', 
                                        'nEnergenti':'EnergySources', 'vrednost':'Amount'})
#Nan Imputing
energetics = energetics.fillna(0)

#Translating
use = [['Примарна производња енергије', 'Primary Energy Production'], ['Увоз', 'Import'], 
       ['Извоз', 'Export'], ['Салдо залиха', 'Inventory Balance'],
       ['Укупно расположива енергија', 'Total Energy Available'], 
       ['Статистичка разлика', 'Statistical Difference'],
       ['Утрошак за производњу енергије', 'Energy Production Cost'],
       ['Производња енергије трансформацијом', 'Generation of Energy by Transformation'], 
       ['Размена', 'Exchange'], ['Размењени производи', 'Exchanged Products'], 
       ['Интерна размена производа', 'Internal Product Exchange'],
       ['Враћено из петрохемије', 'Returned from Petrochemistry'],
       ['Сопствена потрошња у енергетском сектору', 'Own Consumption in Energy Sector'], 
       ['Губици', 'Losses'],
       ['Енергија расположива за финалну потрошњу', 'Energy Available for Final Consumption'],
       ['Финална потрошња за неенергетске сврхе', 'Final Consumption for Non-energy Purposes']]
for i in use:
    energetics.loc[energetics['Use']==i[0], 'Use'] = i[1]

energy_sources = [['Електрична енергија (укупно), GWh', 'Total Electrical Energy (GWh)'],
                  ['Антрацит, t', 'Anthracite (t)'], 
                  ['Остали битуменозни угаљ, t', 'Other Bituminous Coal (t)'],
                  ['Суб-битуменозни угаљ, мрки угаљ и лигнит, t', 'Sub-bituminous Coal, Brown Coal and Lignite (t)'],
                  ['Брикет каменог угља, t', 'Coal Briquette (t)'],
                  ['Брикет мрког угља и лигнита, сушени лигнит, t', 'Briquette of Brown Coal and Lignite, Dried Lignite (t)'],
                  ['Катран од угља, t', 'Coal Tar (t)'], ['Кокс, t', 'Coke (t)'], 
                  ['Високо-пећни гас, 000 m3', 'Blast Furnace Gas (thousands of m3)'],
                  ['Сирова нафта, t', 'Crude Oil (t)'], ['Течности природног гаса, t', 'Natural Gas Liquids (t)'],
                  [' Рафинисана основна сировина, t', 'Refined Raw Material (t)'], ['Адитиви, t', 'Additives (t)'],
                  ['Остали угљоводоници, t', 'Other Hydrocarbons (t)'], ['Рафинеријски гас, t', 'Refinery Gas (t)'],
                  ['Течни нафтни гас, t', 'LPG (t)'], ['Нафта, t', 'Oil (t)'], 
                  ['Безоловни моторни бензин, t', 'Unleaded Motor Gasoline (t)'],
                  ['Оловни бензин, t', 'Lead Gasoline (t)'], ['Авионски бензин, t', 'Plane Gasoline (t)'],
                  ['Гориво за млазне моторе керозинског типа, t', 'Kerosene Type Jet Engine Fuel (t)'],
                  ['Остали керозин, t', 'Other Kerosene (t)'],
                  ['Дизел, t', 'Diesel (t)'], ['Гориво за ложење и остала гасна уља, t', 'Fuel Oil and Other Gas Oils (t)'],
                  ['Уље за ложење (мазут) S≥1%, t', 'Mazut S≥1% (t)'], 
                  ['Уље за ложење (мазут) S<1%, t', 'Mazut S<1% (t)'],
                  ['Специјални бензини, t', 'Special Gasoline (t)'], ['Мазива, t', 'Lubricants (t)'], 
                  ['Битумен, t', 'Bitumen (t)'],
                  ['Парафински восак, t', 'Paraffin wax (t)'], ['Нафтни кокс, t', 'Petroleum Coke (t)'],
                  ['Остали деривати нафте, t', 'Other Petroleum Products (t)'], ['Природни гас, 000 Stm3', 'Natural Gas (thousands of m3)'],
                  ['Огревно дрво, t', 'Firewood (t)'], ['Дрвни остатак и дрвна сечка, t', 'Wood Residue and Wood Chips (t)'],
                  ['Дрвни брикети, t', 'Wood Briquettes (t)'], ['Дрвни пелети, t', 'Wood Pellets (t)'], 
                  ['Дрвени угаљ, t', 'Charcoal (t)'], ['Биогас, 000 m3', 'Biogas (thousands of m3)']]
for i in energy_sources:
    energetics.loc[energetics['EnergySources']==i[0], 'EnergySources'] = i[1]
    
datasets.append([energetics, 'energetics'])

## 8. Use of Information and Communication Technologies

In [187]:
#Percent of Presence of Certain Devices within Households
household_devices = pd.read_csv('08devices_present_in_households.csv', sep=';')

#Renaming and dropping columns
household_devices = household_devices.drop(['idindikator', 'nTer', 'IDTer', 'IDVrUredjaja', 'mes'], axis=1)
household_devices = household_devices.rename(columns={'god':'Year', 'nVrUredjaja':'Device', 
                                                      'vrednost':'PctPresence'})
#Translating
devices = [['ТВ', 'TV'], ['Мобилни телефон', 'Mobile Phone'], ['Лаптоп', 'Laptop'],
           ['Персонални рачунар (PC)', 'PC'], ['Кабловска ТВ', 'Cable TV']]
for i in devices:
    household_devices.loc[household_devices['Device']==i[0], 'Device'] = i[1]
    
#Sorting and Imputing
household_devices = household_devices.sort_values(['Year', 'Device']).reset_index(drop=True)
household_devices = household_devices.fillna(0)


#Percent of Enterprises with a Website by Region
enterp_website = pd.read_csv('08enterprises_with_website.csv', sep=';')

#Renaming and dropping columns
enterp_website = enterp_website.drop(['idindikator', 'IDTer', 'mes'], axis=1)
enterp_website = enterp_website.rename(columns={'god':'Year', 'nTer':'Region', 
                                                'vrednost':'PctPresence'})
#Translating
for i in regions:
    enterp_website.loc[enterp_website['Region']==i[0], 'Region'] = i[1]
    
#Sorting and Imputing
enterp_website = enterp_website.sort_values(['Region', 'Year']).reset_index(drop=True)


#Internet Use Frequency by Individuals
internet_use_freq = pd.read_csv('08individual_internet_use_freq.csv', sep=';')

#Renaming and dropping columns
internet_use_freq = internet_use_freq.drop(['idindikator', 'IDTer', 'nTer', 'mes', 'IDUpotrebaIKT'], axis=1)
internet_use_freq = internet_use_freq.rename(columns={'god':'Year', 'nUpotrebaIKT':'FreqChoice', 
                                                'vrednost':'PctPresence'})
#Translating
choices = [['Никада није користио/користила', 'Never Used'], 
           ['У последња 3 месеца', '< 3 Months'],
           ['Пре више од 3 месеца (мање од 1 године)', '> 3 Months, < 1 Year'],
           ['Пре више од годину дана', '> 1 Year']]
for i in choices:
    internet_use_freq.loc[internet_use_freq['FreqChoice']==i[0], 'FreqChoice'] = i[1]
    
#Sorting and Imputing
internet_use_freq = internet_use_freq.sort_values(['Year', 'FreqChoice']).reset_index(drop=True)

datasets.append([household_devices, 'household_devices'])
datasets.append([enterp_website, 'enterp_website'])
datasets.append([internet_use_freq, 'internet_use_freq'])

## 9. Research & Development

In [188]:
#R&D Percent in GDP
rd_pct_gdp = pd.read_csv('09r&d_pct_gdp.csv', sep=';')

#Renaming and dropping columns
rd_pct_gdp = rd_pct_gdp.drop(['idindikator', 'IDTer', 'nTer', 'mes'], axis=1)
rd_pct_gdp = rd_pct_gdp.rename(columns={'god':'Year', 'vrednost':'GDPPct'})


#Generated Funds Amount
rd_generated_funds = pd.read_csv('09r&d_generated_funds.csv', sep=';')

#Renaming and dropping columns
rd_generated_funds = rd_generated_funds.drop(['idindikator', 'IDTer', 'IDSredstva', 'mes'], axis=1)
rd_generated_funds = rd_generated_funds.rename(columns={'god':'Year', 'vrednost':'Amount(RSD)',
                                        'nSredstva':'FundSource', 'nTer':'Region'})
#Translating
for i in regions:
    rd_generated_funds.loc[rd_generated_funds['Region']==i[0], 'Region'] = i[1]

funds = [['Укупна средства', 'Total Funds'],
         ['Средства обезбеђена из буџета', 'Funds from Budget']]
for i in funds:
    rd_generated_funds.loc[rd_generated_funds['FundSource']==i[0], 'FundSource'] = i[1]
    
#Dropping Rows
rd_generated_funds = rd_generated_funds.loc[rd_generated_funds['Region'].isin(smaller_regions)]
rd_generated_funds = rd_generated_funds.reset_index(drop=True)

#Imputation
rd_generated_funds = rd_generated_funds.fillna(0)

datasets.append([rd_pct_gdp, 'rd_pct_gdp'])
datasets.append([rd_generated_funds, 'rd_generated_funds'])

## 10. Education (HS)

In [189]:
high_school = pd.read_csv('10high_school_graduated.csv', sep=';')
high_school = high_school.loc[high_school['nTer']=='РЕПУБЛИКА СРБИЈА']

#Renaming and dropping columns
high_school = high_school.drop(['idindikator', 'IDTer', 'nTer', 'IDPodrucjaRada',
                                'IDPol'], axis=1)
high_school = high_school.rename(columns={'IDSkolskaGodina':'SchoolYear', 'nPol':'Gender',
                                          'vrednost':'Students', 'nPodrucjaRada':'Programme'})
#Translating
programmes = [['УКУПНО', 'Total'], ['ГИМНАЗИЈА', 'General-education HS'], 
              ['ПОЉОПРИВРЕДА, ПРОИЗВОДЊА И ПРЕРАДА ХРАНЕ', 'Agriculture and Good Production'],
              ['ШУМАРСТВО И ОБРАДА ДРВЕТА', 'Forestry and Wood Processing'], 
              ['ГЕОЛОГИЈА, РУДАРСТВО И МЕТАЛУРГИЈА', 'Geology, Mining and Metallurgy'],
              ['МАШИНСТВО И ОБРАДА МЕТАЛА', 'Machinery and Metal Processing'], 
              ['ЕЛЕКТРОТЕХНИКА', 'Electrical Engineering'],
              ['ХЕМИЈА, НЕМЕТАЛИ И ГРАФИЧАРСТВО', 'Chemistry, Graphene and Non-metals'], 
              ['ТЕКСТИЛСТВО И КОЖАРСТВО', 'Textile and Learher Processing'],
              ['ГЕОДЕЗИЈА И ГРАЂЕВИНАРСТВО', 'Geodesy and Construction'], 
              ['САОБРАЋАЈ', 'Transportation'],
              ['ТРГОВИНА, УГОСТИТЕЉСТВО И ТУРИЗАМ', 'Trade, Hotels and Tourism'],
              ['ЕКОНОМИЈА, ПРАВО И АДМИНИСТРАЦИЈА', 'Economy, Law and Administration'], 
              ['ХИДРОМЕТЕОРОЛОГИЈА', 'Hydrometeorology'],
              ['КУЛТУРА, УМЕТНОСТ И ЈАВНО ИНФОРМИСАЊЕ', 'Culutre, Arts and Public Information'],
              ['ЗДРАВСТВО И СОЦИЈАЛНА ЗАШТИТА', 'Health and Social Welfare'], 
              ['ОСТАЛО (ЛИЧНЕ УСЛУГЕ)', 'Other (Personal Services)'],
              ['ВОЈНЕ ШКОЛЕ', 'Military School']]
for i in programmes:
    high_school.loc[high_school['Programme']==i[0], 'Programme'] = i[1]
    
for i in genders:
    high_school.loc[high_school['Gender']==i[0], 'Gender'] = i[1]
    
datasets.append([high_school, 'high_school'])

## 11. Traffic, Transport and Telecommunications

In [190]:
passengers_and_goods = pd.read_csv('11passengers_goods_trans.csv', sep=';')

#Renaming and dropping columns
passengers_and_goods = passengers_and_goods.drop(['idindikator', 'IDTer', 'nTer', 
                                                  'IDVrPod', 'mes'], axis=1)
passengers_and_goods = passengers_and_goods.rename(columns={'nVrPod':'Metric', 'god':'Year',
                                                            'vrednost':'Value'})
#Translating
metrics = [['број превезених путника, хиљ.', 'Passengers Carried (Thousands)'], 
           ['путнички километри, мил.', 'Passenger Kilometers (Millions)'],
           ['превезена роба, t', 'Goods Transported (Tons)'], 
           ['тонски километри, мил.', 'Ton Kilometers (Millions)']]
for i in metrics:
    passengers_and_goods.loc[passengers_and_goods['Metric']==i[0], 'Metric'] = i[1]
    

road_traffic = pd.read_csv('11road_traffic.csv', sep=';')

#Renaming and dropping columns
road_traffic = road_traffic.drop(['idindikator', 'IDVrPod', 'mes'], axis=1)
road_traffic = road_traffic.rename(columns={'nVrPod':'VehicleType', 'god':'Year',
                                                            'vrednost':'Amount'})
#Translating
vehicles = [['Мотоцикли', 'Motorcycles'], ['Путнички аутомобили', 'Passenger Cars'], 
            ['Специјална путничка возила', 'Special Passenger Vehicles'], ['Аутобуси', 'Buses'], 
            ['Теретна возила', 'Trucks'], 
            ['Специјална теретна возила', 'Special Lorries'],
            ['Радна возила', 'Working Purpose Vehicles'], ['Вучна возила', 'Towing Vehicles'], 
            ['Прикључна возила', 'Trailers']]
for i in vehicles:
    road_traffic.loc[road_traffic['VehicleType']==i[0], 'VehicleType'] = i[1]
    
#Vehicle Groups with too Little Information
drop_vehicles = ['Special Passenger Vehicles', 'Special Lorries']
road_traffic = road_traffic[~road_traffic['VehicleType'].isin(drop_vehicles)]
road_traffic = road_traffic.reset_index(drop=True)


air_traffic = pd.read_csv('11air_traffic.csv', sep=';')
air_traffic = air_traffic.loc[air_traffic['nVrPod']!='Деонички летови'].reset_index(drop=True)

#Renaming and dropping columns
air_traffic = air_traffic.drop(['idindikator', 'IDVrPod', 'mes'], axis=1)
air_traffic = air_traffic.rename(columns={'nVrPod':'Metric', 'god':'Year', 'vrednost':'Amount'})

#Translating
air_metrics = [['Авио km, хиљ.', 'Air Kilometers (Thousands)'], 
               ['Часови летења', 'Flight Hours'],
               ['Превезени путници, хиљ.', 'Passengers Carried (Thousands)'], 
               ['Путнички километри, мил.', 'Passenger Kilometers (Millions)'],
               ['Превезени терет, t', 'Freight (Tons)'], ['Запослени', 'Employees'],
               ['Тонски километри терета, хиљ.', 'Freight Tonne-kilometers (Thousands)'],
               ['Утрошак горива, t', 'Fuel Consumption (Tones)']]
for i in air_metrics:
    air_traffic.loc[air_traffic['Metric']==i[0], 'Metric'] = i[1]
    
#Imputing Fuel Consumption
#Plot
#import matplotlib.pyplot as plt

hours = air_traffic.loc[air_traffic['Metric']=='Flight Hours']['Amount'][1:]
fuel = air_traffic.loc[air_traffic['Metric']=='Fuel Consumption (Tones)']['Amount'][1:]

#plt.scatter(x=hours, y=fuel)
#plt.xlabel('Flight Hours')
#plt.ylabel('Fuel Consumption (Tones)')
#plt.show()

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(hours.values.reshape(-1, 1), fuel.values.reshape(-1, 1))
fill_value = lr.predict(air_traffic.iloc[1, 2].reshape(1, -1))[0][0]

air_traffic = air_traffic.fillna(fill_value)

datasets.append([passengers_and_goods, 'passengers_and_goods'])
datasets.append([road_traffic, 'road_traffic'])
datasets.append([air_traffic, 'air_traffic'])

## 12. Job Market

In [5]:
average_salaries = pd.read_json('12average_salaries.json')
average_salaries = average_salaries.loc[(average_salaries['mes']==0)&(average_salaries['nVrPod']=='нето зараде')&(average_salaries['nTer']=='РЕПУБЛИКА СРБИЈА')].reset_index(drop=True)

#Renaming and dropping columns
average_salaries = average_salaries.drop(['idindikator', 'IDVrPod', 'mes', 'nVrPod', 'IDTer',
                                          'nTer', 'IDKD08'], axis=1)
average_salaries = average_salaries.rename(columns={'nkd08':'Sector', 'god':'Year', 
                                                    'vrednost':'Amount(RSD)'})
#Translating
sectors = [['Укупно', 'Total'], 
           ['A - Пољопривреда, шумарство и рибарство', 'Agriculture, Forestry and Fishing'],
           ['B - Рударство', 'Mining'], 
           ['C - Прерађивачка индустрија', 'Manufacturing Industry'],
           ['D - Снабдевање електричном енергијом, гасом, паром и климатизација', 'Electricity, Gas, Steam and Air Conditioning Supply'],
           ['E - Снабдевање водом; управљање отпадним водама, контролисање процеса уклањања отпада и сличне активности', 'Waterworks'],
           ['F - Грађевинарство', 'Construction'],
           ['G - Трговина на велико и трговина на мало; поправка моторних возила и мотоцикала', 'Wholesale and Retail Trade, Repair of Vehicles'],
           ['H - Саобраћај и складиштење', 'Transport, Traffic and Storage'], 
           ['I - Услуге смештаја и исхране', 'Accommodation and Catering Services'],
           ['J - Информисање и комуникације', 'Information and Communications'],
           ['K - Финансијске делатности и делатност осигурања', 'Financial and Insurance Activities'],
           ['L - Пoсловање некретнинама', 'Real Estate Business'],
           ['M - Стручне, научне и техничке делатности', 'Professional, Scientific and Technical Activities'],
           ['N - Административне и помоћне услужне делатности', 'Administrative and Support Services'],
           ['O - Државна управа и одбрана; обавезно социјално осигурање', 'State Administration and Defence; Compulsory Social Security'],
           ['P - Образовање', 'Education'], 
           ['Q - Здравствена и социјална заштита', 'Health and Social Care'],
           ['R - Уметност; забава и рекреација', 'Art; Entertainment and Recreation'],
           ['S - Остале услужне делатности', 'Other Services']]
for i in sectors:
    average_salaries.loc[average_salaries['Sector']==i[0], 'Sector'] = i[1]
    
    
#NEET rate - 15-29
NEET_rate = pd.read_csv('12NEET_rate.csv', sep=';')
NEET_rate = NEET_rate.loc[(NEET_rate['nStarGrupa']=='15-29')&(NEET_rate['nPol']=='Укупно')]
NEET_rate = NEET_rate.reset_index(drop=True)

#Renaming and dropping columns
NEET_rate = NEET_rate.drop(['idindikator', 'mes', 'IDTer', 'nTer', 'IDStarGrupa',
                            'IDPol', 'nPol', 'nStarGrupa'], axis=1)
NEET_rate = NEET_rate.rename(columns={'god':'Year', 'vrednost':'PctUnemployed'})


#Unemployment Rate
unemployed = pd.read_csv('12unemployed.csv', sep=';')
unemployed = unemployed.loc[(unemployed['nTer']=='РЕПУБЛИКА СРБИЈА')&(unemployed['nPol']=='Укупно')&(unemployed['nStarGrupa']=='Лица радног узраста (15-64)')].reset_index(drop=True)

#Renaming and dropping columns
unemployed = unemployed.drop(['idindikator', 'IDTer', 'nTer', 'IDStarGrupa',
                              'IDPol', 'nPol', 'nStarGrupa'], axis=1)
unemployed = unemployed.rename(columns={'god':'Year', 'vrednost':'PctUnemployed', 'mes':'Q'})

#Translating
for i in qs:
    unemployed.loc[unemployed['Q']==i[0], 'Q'] = i[1]
    
#Sorting
unemployed = unemployed.sort_values(['Year', 'Q']).reset_index(drop=True)

datasets.append([average_salaries, 'average_salaries'])
datasets.append([NEET_rate, 'NEET_rate'])
datasets.append([unemployed, 'unemployed'])

## 13. Structural Business Statistics

In [192]:
#Number of Enterprises
enterprises = pd.read_json('13number_of_enterprises.json')
enterprises = enterprises.loc[(enterprises['nTer']=='РЕПУБЛИКА СРБИЈА')&(enterprises['VelP']=='УКУПНО')].reset_index(drop=True)

#Renaming and dropping columns
enterprises = enterprises.drop(['idindikator', 'IDTer', 'nTer', 'IDKD08',
                              'IDNivoVelP', 'VelP', 'mes'], axis=1)
enterprises = enterprises.rename(columns={'god':'Year', 'vrednost':'Amount', 'nkd08':'Activity'})

#Dropping Rows
relevant_sectors = np.append(enterprises['Activity'].unique()[-14:-2], [(enterprises['Activity'].unique()[-1]), 'Укупно'])
enterprises = enterprises.loc[enterprises['Activity'].isin(relevant_sectors)].reset_index(drop=True)

#Translating
for i in sectors:
    enterprises.loc[enterprises['Activity']==i[0], 'Activity'] = i[1]
    

#Number of Employees
employees = pd.read_json('13number_of_employees.json')
employees = employees.loc[(employees['nTer']=='РЕПУБЛИКА СРБИЈА')&(employees['VelP']=='УКУПНО')].reset_index(drop=True)

#Renaming and dropping columns
employees = employees.drop(['idindikator', 'IDTer', 'nTer', 'IDKD08',
                              'IDNivoVelP', 'VelP', 'mes'], axis=1)
employees = employees.rename(columns={'god':'Year', 'vrednost':'Amount', 'nkd08':'Activity'})

#Dropping Rows
employees = employees.loc[employees['Activity'].isin(relevant_sectors)].reset_index(drop=True)

#Translating
for i in sectors:
    employees.loc[employees['Activity']==i[0], 'Activity'] = i[1]
    

#Total Added Value by Enterprizes
added_value = pd.read_json('13added_value.json')
added_value = added_value.loc[(added_value['nTer']=='РЕПУБЛИКА СРБИЈА')&(added_value['VelP']=='УКУПНО')].reset_index(drop=True)

#Renaming and dropping columns
added_value = added_value.drop(['idindikator', 'IDTer', 'nTer', 'IDKD08',
                              'IDNivoVelP', 'VelP', 'mes'], axis=1)
added_value = added_value.rename(columns={'god':'Year', 'vrednost':'Amount(mRSD)', 'nkd08':'Activity'})

#Dropping Rows
added_value = added_value.loc[added_value['Activity'].isin(relevant_sectors)].reset_index(drop=True)

#Translating
for i in sectors:
    added_value.loc[added_value['Activity']==i[0], 'Activity'] = i[1]
    
    
#Personnel Costs
costs = pd.read_json('13costs.json')
costs = costs.loc[(costs['nTer']=='РЕПУБЛИКА СРБИЈА')&(costs['VelP']=='УКУПНО')].reset_index(drop=True)

#Renaming and dropping columns
costs = costs.drop(['idindikator', 'IDTer', 'nTer', 'IDKD08',
                              'IDNivoVelP', 'VelP', 'mes'], axis=1)
costs = costs.rename(columns={'god':'Year', 'vrednost':'Amount(mRSD)', 'nkd08':'Activity'})

#Dropping Rows
costs = costs.loc[costs['Activity'].isin(relevant_sectors)].reset_index(drop=True)

#Translating
for i in sectors:
    costs.loc[costs['Activity']==i[0], 'Activity'] = i[1]
    

#Concatenating Costs and Added Value
value_added = pd.merge(added_value, costs, on=['Year', 'Activity'], 
                       suffixes=('ValueAdded', 'PersonnelCost'))
value_added['NetValueAdded(mRSD)'] = value_added['Amount(mRSD)ValueAdded'] - value_added['Amount(mRSD)PersonnelCost']


#Per Employee Average Added Value
pea_added_value = pd.read_json('13per_employee_average_added_value.json')

#Renaming and dropping columns
pea_added_value = pea_added_value.drop(['idindikator', 'IDTer', 'nTer', 'IDKD08', 'mes'], axis=1)
pea_added_value = pea_added_value.rename(columns={'god':'Year', 'vrednost':'Amount(tRSD)', 'nkd08':'Activity'})

#Translating
for i in sectors:
    pea_added_value.loc[pea_added_value['Activity']==i[0], 'Activity'] = i[1]



#Per Employee Average Cost
pea_cost = pd.read_json('13per_employee_average_cost.json')

#Renaming and dropping columns
pea_cost = pea_cost.drop(['idindikator', 'IDTer', 'nTer', 'IDKD08', 'mes'], axis=1)
pea_cost = pea_cost.rename(columns={'god':'Year', 'vrednost':'Amount(tRSD)', 'nkd08':'Activity'})

#Translating
for i in sectors:
    pea_cost.loc[pea_cost['Activity']==i[0], 'Activity'] = i[1]
    

#Concatenating Per Employee Added Value and Cost
pea_value_added = pd.merge(pea_added_value, pea_cost, on=['Year', 'Activity'], 
                       suffixes=('ValueAdded', 'PersonnelCost'))
pea_value_added['NetValueAdded(tRSD)'] = pea_value_added['Amount(tRSD)ValueAdded'] - pea_value_added['Amount(tRSD)PersonnelCost']

datasets.append([enterprises, 'enterprises'])
datasets.append([employees, 'employees'])
datasets.append([value_added, 'value_added'])
datasets.append([pea_value_added, 'pea_value_added'])

## Exporting

In [193]:
os.chdir('C:\\Users\\vucin\\Desktop\\The problem with Serbian economy\\2.EDA')

for i in datasets:
    i[0].to_csv(i[1]+'.csv', index=False)